In [ ]:
# !pip install pandas


[notice] A new release of pip is available: 24.3.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip



  Using cached pandas-3.0.0-cp313-cp313-win_amd64.whl.metadata (19 kB)
  Using cached tzdata-2025.3-py2.py3-none-any.whl.metadata (1.4 kB)
   ---------------------------------------- 0.0/9.7 MB ? eta -:--:--
   --- ------------------------------------ 0.8/9.7 MB 7.5 MB/s eta 0:00:02
   ---------- ----------------------------- 2.6/9.7 MB 7.8 MB/s eta 0:00:01
   ------------------- -------------------- 4.7/9.7 MB 8.7 MB/s eta 0:00:01
   ------------------------------ --------- 7.3/9.7 MB 9.9 MB/s eta 0:00:01
   ---------------------------------------- 9.7/9.7 MB 10.8 MB/s eta 0:00:00
Using cached tzdata-2025.3-py2.py3-none-any.whl (348 kB)


In [3]:
import sqlite3
import pandas as pd
from pathlib import Path

# Caminho do seu banco de dados (confirme se é este mesmo)
db_path = Path(r"G:\Meu Drive\4. Registros\Glaydson\Orçamento\db\finance_abs.db")

if not db_path.exists():
    print(f"❌ Erro: Banco de dados não encontrado em: {db_path}")
    print("Verifique se o Google Drive está conectado ou ajuste o caminho.")
else:
    print(f"✅ Banco encontrado: {db_path}")
    conn = sqlite3.connect(db_path)

✅ Banco encontrado: G:\Meu Drive\4. Registros\Glaydson\Orçamento\db\finance_abs.db


In [4]:
# Busca tudo que veio de arquivos CSV (Conta Corrente)
query_check = "SELECT * FROM transactions WHERE source LIKE '%OUROCARD%'"
df_check = pd.read_sql_query(query_check, conn)

print(f"📉 Total de lançamentos de Conta Corrente encontrados: {len(df_check)}")

if not df_check.empty:
    print("\n🔍 Amostra dos dados que SERÃO APAGADOS:")
    display(df_check.head())
    
    # Verifica se existem dados de outras fontes (para garantir que não vamos zerar o banco todo)
    query_others = "SELECT source, count(*) as qtd FROM transactions WHERE source NOT LIKE '%CSV%' GROUP BY source"
    df_others = pd.read_sql_query(query_others, conn)
    print("\n🛡️ Dados que SERÃO PRESERVADOS (Não serão apagados):")
    display(df_others)
else:
    print("Nenhum lançamento de Conta Corrente encontrado.")

📉 Total de lançamentos de Conta Corrente encontrados: 1879

🔍 Amostra dos dados que SERÃO APAGADOS:


,hash_id,date,description,amount,source,category,is_manual
0,cdadd87af5ba39dbea65f93c0bce17aa,2024-03-04,PGTO. COBRANCA 4766 000000200 200300,-3617.11,Card: OUROCARD_ELO_NANQUIM-Abr_24.txt,Cartão de Crédito,0
1,0231444f384fabdb0dbdce7ba62f5dac,2024-03-02,CIRANDA KIDS BRASILIA BR,-160.00,Card: OUROCARD_ELO_NANQUIM-Abr_24.txt,Cartão de Crédito,0
2,a342e2c156b8c0a91725a76f2ab7b222,2024-03-16,CINEMARK TAGUATINGA BRASILIA BR,-102.50,Card: OUROCARD_ELO_NANQUIM-Abr_24.txt,Cartão de Crédito,0
3,847bc5156e4769dfa335421f327e4e6f,2024-03-24,CINEMARK TAGUATINGA BRASILIA BR,-70.00,Card: OUROCARD_ELO_NANQUIM-Abr_24.txt,Cartão de Crédito,0
4,8f5e96c4ce6fd7f77b87022a59fbf9df,2024-03-02,LUMAFAST RESTAURANTES BRASILIA BR,-36.50,Card: OUROCARD_ELO_NANQUIM-Abr_24.txt,Cartão de Crédito,0



🛡️ Dados que SERÃO PRESERVADOS (Não serão apagados):


,source,qtd
0,Card: OUROCARD_ELO_NANQUIM-Abr_24.txt,52
1,Card: OUROCARD_ELO_NANQUIM-Abr_25.txt,55
2,Card: OUROCARD_ELO_NANQUIM-Ago_24.txt,43
3,Card: OUROCARD_ELO_NANQUIM-Ago_25.txt,25
4,Card: OUROCARD_ELO_NANQUIM-Dez_24.txt,42
5,Card: OUROCARD_ELO_NANQUIM-Dez_25.txt,5
6,Card: OUROCARD_ELO_NANQUIM-Fev_25.txt,50
7,Card: OUROCARD_ELO_NANQUIM-Jan_25.txt,43
8,Card: OUROCARD_ELO_NANQUIM-Jan_26.txt,12
9,Card: OUROCARD_ELO_NANQUIM-Jul_24.txt,48


In [6]:
if not df_check.empty:
    cursor = conn.cursor()
    
    # Executa a remoção
    cursor.execute("DELETE FROM transactions WHERE source LIKE '%CSV%'")
    conn.commit()
    
    deleted_count = cursor.rowcount
    print(f"🗑️ Sucesso! {deleted_count} registros foram removidos permanentemente.")
else:
    print("Nada para apagar.")

🗑️ Sucesso! 3100 registros foram removidos permanentemente.


In [7]:
# Prova real
remaining = pd.read_sql_query("SELECT count(*) FROM transactions WHERE source LIKE '%CSV%'", conn).iloc[0,0]

if remaining == 0:
    print("✅ Limpeza confirmada: 0 registros de CSV restantes.")
    
    # Mostra o que sobrou no banco
    final_df = pd.read_sql_query("SELECT source, count(*) as qtd FROM transactions GROUP BY source", conn)
    display(final_df)
else:
    print(f"⚠️ Atenção: Ainda restam {remaining} registros.")

conn.close()

✅ Limpeza confirmada: 0 registros de CSV restantes.


,source,qtd
0,Card: OUROCARD_ELO_NANQUIM-Abr_24.txt,52
1,Card: OUROCARD_ELO_NANQUIM-Abr_25.txt,55
2,Card: OUROCARD_ELO_NANQUIM-Ago_24.txt,43
3,Card: OUROCARD_ELO_NANQUIM-Ago_25.txt,25
4,Card: OUROCARD_ELO_NANQUIM-Dez_24.txt,42
5,Card: OUROCARD_ELO_NANQUIM-Dez_25.txt,5
6,Card: OUROCARD_ELO_NANQUIM-Fev_25.txt,50
7,Card: OUROCARD_ELO_NANQUIM-Jan_25.txt,43
8,Card: OUROCARD_ELO_NANQUIM-Jan_26.txt,12
9,Card: OUROCARD_ELO_NANQUIM-Jul_24.txt,48


In [1]:
### CONSULTAR CONTEÚDO DO BANCO DE DADOS ###
import sqlite3
import pandas as pd
from pathlib import Path

# Configuração do Caminho (O mesmo que configuramos no sistema)
db_path = Path(r"G:\Meu Drive\4. Registros\Glaydson\Orçamento\db\finance_abs.db")

if not db_path.exists():
    print(f"❌ Banco não encontrado em: {db_path}")
    print("Verifique se o Google Drive está conectado.")
else:
    conn = sqlite3.connect(db_path)
    print(f"✅ Conectado com sucesso!")

    # 1. Visão Geral (Últimos registros inseridos)
    print("\n--- 🔍 Últimas 10 Transações Inseridas ---")
    query_recent = "SELECT date, description, amount, source, category FROM transactions ORDER BY date DESC LIMIT 10"
    df_recent = pd.read_sql_query(query_recent, conn)
    display(df_recent)

✅ Conectado com sucesso!

--- 🔍 Últimas 10 Transações Inseridas ---


,date,description,amount,source,category
0,2025-12-29,LIVRARIA BOOK PARC 01/12 SAO PAULO BR,-70.12,Card: OUROCARD_ELO_NANQUIM-Jan_26.txt,Cartão de Crédito
1,2025-12-23,FF BISCOITOS CASEIROS BRASILIA BR,-37.00,Card: OUROCARD_VISA_INFINITE-Jan_26.txt,Cartão de Crédito
2,2025-12-21,ESTAC SHOPPING BRASILIA BR,-25.00,Card: OUROCARD_VISA_INFINITE-Jan_26.txt,Cartão de Crédito
3,2025-12-18,GAZETA DO PO*Gaze CURITIBA BR,-1.90,Card: OUROCARD_VISA_INFINITE-Jan_26.txt,Cartão de Crédito
4,2025-12-17,BRASIL TEMPER BRASILIA BR,-243.60,Card: OUROCARD_ELO_NANQUIM-Jan_26.txt,Cartão de Crédito
5,2025-12-17,CASA FESTA PARC 01/02 BRASILIA BR,-144.60,Card: OUROCARD_ELO_NANQUIM-Jan_26.txt,Cartão de Crédito
6,2025-12-14,EVOLVEAGUASCLARAS BRASILIA BR,-119.90,Card: OUROCARD_VISA_INFINITE-Jan_26.txt,Cartão de Crédito
7,2025-12-13,ACADEMIA VASCO NETO BRASILIA BR,-269.00,Card: OUROCARD_ELO_NANQUIM-Jan_26.txt,Cartão de Crédito
8,2025-12-12,Google Play Books SAO PAULO BR,-0.99,Card: OUROCARD_VISA_INFINITE-Jan_26.txt,Cartão de Crédito
9,2025-12-11,AMAZONMKTPLC JACKELINE SAO PAULO BR,-186.03,Card: OUROCARD_ELO_NANQUIM-Jan_26.txt,Cartão de Crédito


In [ ]:
### EXCLUIR CONTEÚDOS ESPECÍFICOS DO BANCO DE DADOS ###

import sqlite3
import pandas as pd
from pathlib import Path

# --- CONFIGURAÇÃO ---
# Digite aqui o que você quer procurar para apagar
TERMO_BUSCA = input("Qual termo devo procurar para selecionar as linhas a serem excluídas? ")

# Caminho do banco
db_path = Path(r"G:\Meu Drive\4. Registros\Glaydson\Orçamento\db\finance_abs.db")

if not db_path.exists():
    print("❌ Banco não encontrado. Verifique o caminho.")
else:
    conn = sqlite3.connect(db_path)
    
    # Busca em DESCRIÇÃO ou FONTE (Source)
    # O comando LIKE '%...%' procura o texto em qualquer parte da frase
    query = f"""
        SELECT * FROM transactions 
        WHERE description LIKE '%{TERMO_BUSCA}%' 
           OR source LIKE '%{TERMO_BUSCA}%'
    """
    
    df_to_delete = pd.read_sql_query(query, conn)
    
    print(f"🔎 Procurando por: '{TERMO_BUSCA}'")
    print(f"📉 Registros encontrados para exclusão: {len(df_to_delete)}")
    
    if not df_to_delete.empty:
        print("\nAmostra do que será apagado:")
        display(df_to_delete.head())
    else:
        print("Nenhum registro encontrado com esse termo.")

In [ ]:
a = input("Deseja realmente apagar esses registros? (s/n): ")

print(a)